In [3]:
!pip install -q recsim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
import gym
from recsim import document
from recsim.choice_model import MultinomialLogitChoiceModel
from recsim.simulator import environment
from recsim.simulator import recsim_gym

/usr/local/lib/python3.10/dist-packages/gin/tf/__init__.py:48: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
from geopy.geocoders import Nominatim
from math import radians, sin, cos, sqrt, atan2

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
tourist_spots_data = pd.read_csv("data.csv")
tourist_spots_data

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [ ]:
#tourist_spots_data['Opening Time'] = pd.to_datetime(tourist_spots_data['Opening Time']).dt.time
#tourist_spots_data['Closing Time'] = pd.to_datetime(tourist_spots_data['Closing Time']).dt.time
#tourist_spots_data

In [ ]:
tourist_spots_data.info()

In [ ]:
tourist_spots_data['Budget (INR)'] = tourist_spots_data['Budget (INR)'].astype(int)
tourist_spots_data.info()

In [ ]:
feedback_data = pd.read_csv('feedback.csv')
feedback_data

In [ ]:
feedback_data.info()

In [ ]:
tourist_spots_data['City'].unique()

In [ ]:
def get_location_coordinates(location_name):
    loc = Nominatim(user_agent="GetLoc")
    get_loc = loc.geocode(location_name)
    return get_loc.latitude, get_loc.longitude

In [ ]:
def calculate_distance(lat1, lon1, lat2, lon2):
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    R = 6371.0

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance


In [ ]:
def store_feedback(user_id, place, feedback, rating, date):
    feedback_data = pd.DataFrame({
        'user_id': [user_id],
        'place': [place],
        'feedback': [feedback],
        'rating': [rating],
        'date_of_feedback': [date]
    })
    feedback_data.to_csv("feedback.csv", mode='a', header=False, index=False)

In [ ]:
datetime.now().time()

In [1]:
import gym
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic

class TravelEnv(gym.Env):
    def __init__(self, tourist_spots_data):
        super(TravelEnv, self).__init__()
        self.tourist_spots_data = tourist_spots_data
        self.current_state = None
        self.done = False
        self.user_budget = None
        self.visited_spots = []

    def step(self, action):
        recommended_spot = self.tourist_spots_data.iloc[action]
        spot_name = recommended_spot['Tourist Spot']
        spot_lat = recommended_spot['Location Lat']
        spot_lon = recommended_spot['Location Long']
        spot_distance = self.calculate_distance(spot_lat, spot_lon)

        # Check if the recommended spot has already been visited
        if spot_name in self.visited_spots:
            reward = -0.1  # Penalty for revisiting a spot
        else:
            # Calculate reward based on distance and budget
            reward = self.calculate_reward(spot_distance)
            self.visited_spots.append(spot_name)

        # Update user budget
        self.user_budget -= recommended_spot['Budget (INR)']

        # Check if episode is done (e.g., user reaches the budget limit)
        if self.user_budget <= 0:
            self.done = True

        return self.current_state, reward, self.done, {}

    def reset(self, user_id, location, budget):
        self.user_id = user_id
        self.user_budget = budget
        self.visited_spots = []
        self.current_state = location
        self.done = False
        return self.current_state

    def calculate_distance(self, spot_lat, spot_lon):
        # Calculate distance between user's location and spot's location
        user_location = (self.current_state[0], self.current_state[1])
        spot_location = (spot_lat, spot_lon)
        return geodesic(user_location, spot_location).kilometers

    def calculate_reward(self, distance):
        # Simple reward system: inverse of distance
        return 1 / distance if distance > 0 else 0

In [2]:
def get_recommendation(user_id, place, budget, tourist_spots_data, user_feedback):

    # Check if the user has any previous feedback
    user_history = user_feedback[user_feedback['user_id'] == user_id]
    user_has_history = not user_history.empty

    # Filter tourist spots based on location
    place_filtered = tourist_spots_data[tourist_spots_data['City'] == place]

    # Filter spots based on budget
    affordable_spots = place_filtered[place_filtered['Budget (INR)'].astype(int) <= budget]

    # Filter spots based on opening hours
    current_time = datetime.now().time()
    open_spots = affordable_spots[affordable_spots.apply(lambda x:
                                                        datetime.strptime(x['Opening Time'], "%H:%M:%S").time() <= current_time <= datetime.strptime(x['Closing Time'], "%H:%M:%S").time(), axis=1)]


    if user_has_history:
        # Merge user's history with feedback from other users
        all_feedback = pd.merge(user_feedback, user_history, how='outer')
        feedback_scores = all_feedback.groupby('tourist_spot')['rating'].mean()
        recommended_spots = open_spots.merge(feedback_scores, left_on='Tourist Spot', right_index=True, how='left')
        recommended_spots = recommended_spots.sort_values(by='rating', ascending=False)
    else:
        # If user is new, recommend based on overall ratings
        recommended_spots = open_spots.sort_values(by='rating', ascending=False)


    if not recommended_spots.empty:
        top_recommendation = recommended_spots.iloc[0]['Tourist Spot']
        return top_recommendation
    else:
        return "No recommendations available within your budget and location."


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def main():
    # Initialize environment and other components
    env = TravelEnv(tourist_spots_data)
    choice_features = {
          "Tourist Spot": 0,
          "City": 1,
          "Location Lat": 2,
          "Location Long": 3,
          "Budget (INR)": 4,
          "Opening Time": 5,
          "Closing Time": 6
      }
    choice_model = MultinomialLogitChoiceModel(choice_features)
    doc = document.AbstractDocument
    recsim_env = recsim_gym.RecSimGymEnv(env, choice_model, doc)

    user_id = int(input("Enter user ID: "))
    location = (float(input("Enter latitude: ")), float(input("Enter longitude: ")))
    budget = float(input("Enter budget (INR): "))
    recommendation = get_recommendation(user_id, place, budget, tourist_spots_data, feedback_data)

    # Reset environment with user's details
    observation = env.reset(user_id, location, budget)

    total_reward = 0
    while not env.done:
        # Perform a step in the environment with the recommended spot
        observation, reward, done, _ = env.step(recommendation)
        total_reward += reward

        # Get the next recommendation
        recommendation = get_recommendation(user_id, place, budget, tourist_spots_data, feedback_data)

    print("Total reward:", total_reward)

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import recsim
from recsim import document
from recsim import user
from recsim.choice_model import MultinomialLogitChoiceModel

# Load your custom dataset
data = pd.read_csv('data.csv')

choice_features = {
        "Tourist Spot": 0,
        "City": 1,
        "Location Lat": 2,
        "Location Long": 3,
        "Budget (INR)": 4,
        "Opening Time": 5,
        "Closing Time": 6
    }

# Define user model
# Using GruUserModel with MultinomialLogitChoiceModel for demonstration
user_model = user.AbstractUserModel(
    candidate_embedding_dim=32,  # Dimensionality of candidate embeddings
    gru_hidden_dim=64,  # Dimensionality of GRU hidden states
    vocab_size=len(data),  # Size of the vocabulary (number of unique items)
    num_candidates=len(data),  # Number of candidate items
    num_features=10,  # Number of features per item
    choice_model=MultinomialLogitChoiceModel(choice_features)
)


# Define candidate generator
candidate_gen = document.StaticDocumentSampler(
    num_candidates=len(data),
    items=data.values.tolist(),
    doc_weights=np.ones(len(data))
)

# Define recommender
class LinearRecommender(tf.keras.Model):
    def __init__(self):
        super(LinearRecommender, self).__init__()
        self.dense = tf.keras.layers.Dense(1)

    def call(self, inputs):
        return self.dense(inputs)

recommender = LinearRecommender()

# Define reward function
# You may need to customize this function based on your dataset and recommendation goals
def reward_fn(doc_obs, user_obs, prev_user_obs):
    # Calculate the reward based on user clicks and document relevance
    click = user_obs['click']  # User click observation
    relevance = doc_obs['relevance']  # Document relevance observation

    # Compute the reward as the sum of clicks multiplied by relevance
    # You may also want to add a scaling factor or other adjustments as needed
    reward = tf.reduce_sum(click * relevance)

    return reward

# Define environment
environment = recsim.environment.Environment(
    user_model=user_model,
    candidate_sampler=candidate_gen,
    doc_sampler=candidate_gen,
    recommender=recommender,
    reward_fn=reward_fn
)

# Train the environment
num_episodes = 1000
batch_size = 32

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

for episode in range(num_episodes):
    user_history, doc_obs, _, _ = environment.reset(num_users)
    user_features = user_history[:, :, 'features']

    for _ in range(batch_size):
        with tf.GradientTape() as tape:
            recommendations = recommender(user_features)
            chosen_docs = environment.step(recommendations.numpy())

            # Compute reward
            reward = reward_fn(doc_obs, user_history[:, :, 'click'], None)

            # Compute loss (for demonstration purposes, using mean squared error)
            loss = tf.reduce_mean(tf.square(reward - chosen_docs))

        # Update model parameters
        gradients = tape.gradient(loss, recommender.trainable_variables)
        optimizer.apply_gradients(zip(gradients, recommender.trainable_variables))

# Evaluate the trained model
# For simplicity, let's assume we want to evaluate the model's performance by calculating the mean reward over a number of episodes
num_evaluation_episodes = 100
total_rewards = []

for _ in range(num_evaluation_episodes):
    user_history, doc_obs, _, _ = environment.reset(num_users)
    user_features = user_history[:, :, 'features']

    episode_reward = 0

    for _ in range(batch_size):
        recommendations = recommender(user_features)
        chosen_docs = environment.step(recommendations.numpy())
        episode_reward += tf.reduce_sum(chosen_docs)

    total_rewards.append(episode_reward)

mean_reward = tf.reduce_mean(total_rewards)
print("Mean Reward:", mean_reward.numpy())


In [ ]:
''